# 🕵️ Forensische Datenanalyse von Weblogs mit DuckLake (DuckDB)Dieses Notebook dokumentiert die forensische Untersuchung von Weblog-Daten zur Identifizierung und zum Beweis einer simulierten **Datenmanipulation**.Wir nutzen **DuckLake** (eine Erweiterung für DuckDB), um die Unveränderlichkeit der Beweiskette (**Chain of Custody**) zu gewährleisten. DuckLake erstellt für jede Schreiboperation einen unveränderlichen **Snapshot** (Version), was das **Time Travel** zur Rekonstruktion des Originalzustands ermöglicht.

---## 1. Zweck und Ziele der Untersuchung### 1.1 UntersuchungszieleZiel der Untersuchung ist es, folgende Fragen anhand der Weblog-Daten zu beantworten:1.  **Beweissicherung:** Erfolgreiche Erfassung des digitalen Beweisstücks (Logdatei) als unveränderlichen **Snapshot 1 (Original)**.2.  **Hypothese der Manipulation:** Simulation und Nachweis der Löschung von Log-Einträgen (Statuscodes >= 400).3.  **Beweisführung:** Wiederherstellung des Originalzustands (Time Travel) und Isolierung der gelöschten Datensätze mittels der Änderungsverfolgung von DuckLake (`ducklake_table_deletions`).### 1.2 Datenakquisition und Setup

In [ ]:
# Installation der notwendigen Bibliotheken (Kaggle für Daten-Download/Annahme)! pip install -q duckdb pandas matplotlib! pip install -q kaggle # Nur nötig, wenn Daten via Kaggle API geladen werden# Erstellung eines Datenverzeichnisses und Entpacken der Log-Daten (Annahme: web-log-dataset.zip ist vorhanden)# **WICHTIG:** Stellen Sie sicher, dass die Datei `weblog.csv` im Verzeichnis `./data/` existiert.! mkdir -p ./data# ! unzip web-log-dataset.zip -d ./data # Führen Sie dies aus, wenn Sie die ZIP-Datei haben# Laden der Bibliotheken und Definition der Pfadeimport osimport timeimport pandas as pdimport duckdbimport shutil# Pfad zur Weblog-Datei (Annahme: weblog.csv ist der Datensatz)LOG_PATH = './data/weblog.csv'# --- Konfiguration der forensischen Datenbankpfade (DuckLake) ---# DuckLake speichert Metadaten und Snapshots in diesen Pfaden.DUCKLAKE_METADATA_PATH = 'forensic_evidence.ducklake'DUCKLAKE_DATA_PATH = 'forensic_data_files'CATALOG_NAME = 'forensic_log_archive'table_name = 'access_logs'# Erstellung der Verzeichnisseos.makedirs(DUCKLAKE_DATA_PATH, exist_ok=True)print(f"✅ Lokale Verzeichnisse für forensische Beweise erstellt.")

### 1.3 Datenbereinigung und Ingestion (Beweissicherung: Snapshot 1)Die Rohdaten werden in Pandas geladen, bereinigt und anschließend in den DuckLake-Katalog als erster unveränderlicher Zustand (*Snapshot 1*) eingefügt. Dies sichert den **Original-Beweis**.

In [ ]:
# --- Datenbereinigung und Transformation ---try:    # 1. Daten einlesen    df = pd.read_csv(LOG_PATH)        # 2. Spaltennamen standardisieren    df.columns = df.columns.str.strip().str.lower()        # 3. Zeitstempel konvertieren (entfernt '[', nutzt Apache Log Format)    df['time'] = pd.to_datetime(        df['time'].astype(str).str.replace("[", "", regex=False),        format="%d/%b/%Y:%H:%M:%S",        errors="coerce"     )        # 4. Spaltenumbenennung ('staus' -> 'status_code' und 'time' -> 'timestamp')    df.rename(columns={'time': 'timestamp', 'staus': 'status_code'}, inplace=True)        # 5. Statuscodes in robusten Integer-Typ konvertieren    df['status_code'] = pd.to_numeric(        df['status_code'],        errors='coerce'     ).astype(pd.Int64Dtype())        # 6. Finale Spaltenauswahl für die Archivierung    df = df[['timestamp', 'ip', 'url', 'status_code']]    print(f"✅ Rohdaten erfolgreich bereinigt. {len(df)} Einträge bereit zur Archivierung.")except FileNotFoundError:    print(f"FEHLER: Datei nicht gefunden unter {LOG_PATH}. Bitte stellen Sie sicher, dass `weblog.csv` im Verzeichnis `./data/` liegt.")    raise# --- DuckLake Initialisierung und Snapshot 1 (Beweissicherung) ---con = duckdb.connect(database='forensic_duckdb.db') # Lokale DuckDB-Datenbankcon.sql("INSTALL ducklake;")con.sql("LOAD ducklake;")# Katalog anhängen und verwenden (definiert den Speicherort der Snapshots)attach_query = f"ATTACH 'ducklake:{DUCKLAKE_METADATA_PATH}' AS {CATALOG_NAME} (DATA_PATH '{DUCKLAKE_DATA_PATH}');"con.sql(attach_query)con.sql(f"USE {CATALOG_NAME};")# 1. Leere Zieltabelle im DuckLake erstellencon.sql(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM df LIMIT 0;")# 2. Daten einfügen (erzeugt Snapshot 1: Original-Beweis)con.sql(f"INSERT INTO {table_name} SELECT * FROM df;")# 3. Snapshot ID 1 zur späteren Time Travel Verwendung speichernsnapshot_1_id = con.sql(f"SELECT max(snapshot_id) FROM ducklake_snapshots('{CATALOG_NAME}')").fetchone()[0]print(f"\n✅ Snapshot 1 (Original-Log) erfasst. ID: {snapshot_1_id}")

---## 2. Analyse auf Anomalien, außergewöhnliche Aktivitäten und Beweise### 2.1 Simulation der Manipulation (Erzeugt Snapshot 2)Wir simulieren, wie ein Angreifer kritische Log-Einträge entfernt und einen Ablenkungseintrag hinzufügt. Diese Aktionen führen zum **Snapshot 2**, dem manipulierten Zustand.

In [ ]:
# Kurze Pause, um einen eindeutigen Zeitstempel für den Snapshot 2 zu gewährleistentime.sleep(1) # --- KRITISCHE AKTION 1: Löschen von Fehlereinträgen (Simulierte Manipulation) ---# Entfernung aller kritischen HTTP-Fehler (4xx und 5xx)deleted_count = con.execute(f"DELETE FROM {table_name} WHERE status_code >= 400;").fetchone()[0]print(f" ❌ {deleted_count} Fehler-Einträge (>= 400) wurden gelöscht (Simulierte Manipulation).")# --- KRITISCHE AKTION 2: Hinzufügen eines Ablenkungseintrags ---new_log_df = pd.DataFrame({ 'timestamp': [pd.to_datetime(time.time(), unit='s')],  'ip': ['203.0.113.5'], 'url': ['/index.html'], 'status_code': [200] })con.sql(f"INSERT INTO {table_name} SELECT * FROM new_log_df;")print(f" ✅ Ein neuer 200-Eintrag wurde hinzugefügt (Ablenkung).")# Snapshot 2 ID speichernsnapshot_2_id = con.sql(f"SELECT max(snapshot_id) FROM ducklake_snapshots('{CATALOG_NAME}')").fetchone()[0]print(f"\n✅ Snapshot 2 (Manipuliertes Log) erfasst. ID: {snapshot_2_id}")

### 2.2 Nachweis der Manipulation: VergleichsanalyseWir nutzen **Time Travel**, um die Statuscodes im **Original-Log (Snapshot 1)** mit dem **Manipulierten Log (Snapshot 2)** zu vergleichen und die Diskrepanz als Anomalie zu identifizieren.

In [ ]:
# --- Analyse 1: Statuscodes im ORIGINAL-LOG (Snapshot 1) ---print("\n--- Zählungen im ORIGINAL-LOG (Snapshot 1) ---")con.sql(f""    SELECT         status_code,         count(*) AS anzahl,        round(count(*) * 100.0 / (SELECT count(*) FROM {CATALOG_NAME}.ducklake_table_at_snapshot('{table_name}', {snapshot_1_id})), 2) AS anteil_prozent    FROM         {CATALOG_NAME}.ducklake_table_at_snapshot('{table_name}', {snapshot_1_id}) -- Wichtige Time Travel Funktion!    GROUP BY         status_code    ORDER BY         anzahl DESC;"").show()# --- Analyse 2: Statuscodes im MANIPULIERTEN LOG (Snapshot 2) ---print("\n--- Zählungen im MANIPULIERTEN LOG (Snapshot 2 - Aktueller Stand) ---")con.sql(f""    SELECT         status_code,         count(*) AS anzahl    FROM         {table_name} -- Hier wird der aktuelle (manipulierte) Zustand abgefragt    GROUP BY         status_code    ORDER BY         anzahl DESC;"").show()# **Interpretation:** Im manipulierten Log (Snapshot 2) fehlen alle Fehler (z.B. 404/400/403), die im Original-Log (Snapshot 1) vorhanden waren. Dies ist der erste Beweis der Anomalie.

### 2.3 Ermittlung der gelöschten Beweise (Kritische Beweisführung)Die Funktion **`ducklake_table_deletions`** ist das Herzstück der forensischen Analyse. Sie erlaubt die Rekonstruktion jeder gelöschten Zeile im Übergang zu einem bestimmten Snapshot. Dies liefert den **unwiderlegbaren Beweis** der Manipulation.

In [ ]:
# --- Forensischer Beweis: Wiederherstellung der GELÖSCHTEN DATENSÄTZE ---print("\n--- FORENSISCHER BEWEIS: Die gelöschten Log-Einträge ---")con.sql(f""    SELECT         deleted_timestamp AS Zeitstempel_der_Loeschung,         status_code,         ip,         url    FROM         {CATALOG_NAME}.ducklake_table_deletions('{table_name}')    WHERE         snapshot_id = {snapshot_2_id} -- Zeigt alle Löschungen, die den Übergang zu Snapshot 2 verursacht haben    LIMIT 10; -- Zeige die ersten 10 gelöschten Einträge"").show()# --- Statistische Analyse der gelöschten Einträge (Beweis der Absicht) ---print("\n--- Zählung der gelöschten Statuscodes (Beweis der Absicht) ---")con.sql(f""    SELECT         status_code,         count(*) AS anzahl_geloescht    FROM         {CATALOG_NAME}.ducklake_table_deletions('{table_name}')    WHERE         snapshot_id = {snapshot_2_id}    GROUP BY         status_code    ORDER BY         anzahl_geloescht DESC;"").show()# **Interpretation:** Die `ducklake_table_deletions` Tabelle liefert die **vollständigen Daten** der gelöschten Einträge, was die ursprüngliche `DELETE`-Aktion und die Manipulation zweifelsfrei beweist.

### 2.4 Baseline-Analyse: Top IPs und URLs (Snapshot 1)Zusätzlich zur Beweisführung identifizieren wir die Top-Akteure und -Ressourcen im **unveränderten Original-Log (Snapshot 1)**, um eine Baseline für weitere Untersuchungen zu schaffen.

In [ ]:
# --- Analyse 3: Top 10 IP-Adressen (Original-Log: Snapshot 1) ---print("\n--- Top 10 IP-Adressen (Original-Log) ---")con.sql(f""    SELECT         ip,         count(*) AS anzahl    FROM         {CATALOG_NAME}.ducklake_table_at_snapshot('{table_name}', {snapshot_1_id})     GROUP BY         ip    ORDER BY         anzahl DESC    LIMIT 10;"").show()# --- Analyse 4: Top 10 aufgerufene URLs (Original-Log: Snapshot 1) ---print("\n--- Top 10 aufgerufene URLs (Original-Log) ---")con.sql(f""    SELECT         url,         count(*) AS anzahl    FROM         {CATALOG_NAME}.ducklake_table_at_snapshot('{table_name}', {snapshot_1_id})     GROUP BY         url    ORDER BY         anzahl DESC    LIMIT 10;"").show()

---## 3. Dokumentation und Abschluss### 3.1 Unveränderliche Historie (Audit-Log)Die Tabelle `ducklake_snapshots` dient als manipulationssicheres Audit-Log und dokumentiert jede Änderung an den Log-Daten (einschließlich der Ingestion und der Manipulation). Sie bildet die **Chain of Custody** ab.

In [ ]:
# Abfrage der gesamten Snapshot-Historie als Audit-Logcon.sql(f""    SELECT        snapshot_id,        strftime(snapshot_time, '%Y-%m-%d %H:%M:%S') AS snapshot_time,        changes, -- Zeigt die Netto-Änderung in Zeilen (+/-)        total_rows    FROM        ducklake_snapshots('{CATALOG_NAME}')    ORDER BY        snapshot_id;"").show()# **Erklärung:** Die Historie zeigt:# - **Snapshot 1:** Die ursprüngliche Ingestion (z.B. +16007 Zeilen).# - **Snapshot 2:** Die simulierte Manipulation (zeigt die Nettoänderung aus Löschung und Einfügung, z.B. -251 Zeilen (gelöscht) + 1 Zeile (neu)).

### 3.2 Bereinigung der UmgebungNach Abschluss der forensischen Analyse werden die temporären Datenbank- und Dateisystemobjekte entfernt, um eine saubere Arbeitsumgebung zu gewährleisten.

In [ ]:
# Schließe die DuckDB-Verbindungcon.close()# Entferne die temporär erstellten forensischen Artefakte und Daten (DuckDB- und DuckLake-Dateien)if os.path.exists('forensic_duckdb.db'):    os.remove('forensic_duckdb.db')if os.path.exists(DUCKLAKE_METADATA_PATH):    os.remove(DUCKLAKE_METADATA_PATH)if os.path.exists(DUCKLAKE_DATA_PATH):    shutil.rmtree(DUCKLAKE_DATA_PATH)print("✅ Forensische Umgebung (Datenbank-Dateien und DuckLake-Katalog) erfolgreich bereinigt.")